# MuonDataLib Tutorial 3: Sample Log Filtering


One of the key advantages of event data is the option to create histograms from a subset of the information. Typically this is done based on the values of sample logs. 

## Sample logs

The sample logs will automatically be loaded if they are present in the event nexus file. However, if they are missing it is possible to add them manually. 
In this example we will load some data, then create a pair of sample logs:

- An oscillation in the temperature
- A linear field

The first step is to create the simulated log values

In [ ]:
import numpy as np

def linear(x, m, c):
    return m*x + c

def osc(x, amp, omega, phi):
    return amp*np.sin(omega*x + phi) + amp*1.1

Next we will need to load the data.

In [ ]:
from MuonDataLib.data.loader.load_events import load_events
from MuonDataLib.plot.basic import Figure
import os

file_name = 'HIFI00195790.nxs'
input_file = os.path.join('..', '..', '..', '..', 'test', 'data_files', file_name)
data = load_events(input_file, 64)


To create simulated data that matches the actual data, we need to know when each frame of data starts. The `get_frame_start_times` method provides a list of the start times in seconds.

In [ ]:
frame_start_times = data.get_frame_start_times()
print(frame_start_times)

From the frame start times we can see that the first frame is from about $0.994$ until $1.014$ seconds. We can also see that the last frame starts at about $3.174$ seconds. Now we will create the simulated sample logs (includes random noise), which includes $50$ measurments across the whole collection period.

In [ ]:
from MuonDataLib.data.utils import create_data_from_function

start = frame_start_times[0]
end = frame_start_times[-1]+1
step = (frame_start_times[-1]-frame_start_times[0])/50

x, y = create_data_from_function(start, end, step, [3, 6.1, 0.91], osc, seed=1)
fig = Figure(y_label='Temperature (Kelvin)', x_label='Time (seconds)')
fig.plot(x, y, 'Temp data')
fig.show()

x, y = create_data_from_function(start, end, step, [3.1, 0.1], linear, seed=1)
data.add_sample_log("field", x, y)
fig2 = Figure(y_label='Field (MHz)', x_label='Time (seconds)')
fig2.plot(x, y, 'Field data')
fig2.show()

The sample logs were added by the `data.add_sample_log` command. The arguments are the name of the sample log, the x (time in seconds) values and the y values. Now we have the sample log data we can look at the different types of filters. The two plots above will be helpful for creating sensible filters. Now lets look at some unfiltered histogram data.

In [ ]:
no_filter_hist, bins = data.histogram()
fig = Figure(y_label='Counts')
fig.plot_from_histogram(bins, no_filter_hist, [0])
fig.show()

## Sample log filters - Keeping data above a threshold value

The first type of filter we will look at is one that is directly applied based on a sample log value. To start lets remove all of the data corresponding to field value of less than six. 

In [ ]:
data.keep_data_sample_log_above('field', 6.)

The `keep_data_sample_log_above` command is used to add a filter, the first argument is the sample log name we want to apply the filter to and the second argument is the minimum value we want to keep. Lets compare the resultant histogram with the unfiltered data. 


In [ ]:
hist_above_6, bins = data.histogram()
fig = Figure(y_label='Counts')
fig.plot_from_histogram(bins, no_filter_hist, [0], 'unfiltered, ')
fig.plot_from_histogram(bins, hist_above_6, [0], 'field >=6, ')
fig.show()

We can see that the filter has removed some counts, as expected. However, we may want to check that the filter has behaved as expected. To plot the original and filtered sample log data;

In [ ]:
fig = Figure(y_label='Field (MHz)', x_label='Time (seconds)')

field_log = data.get_sample_log('field')
x0, y0 = field_log.get_original_values()
fig.plot(x0, y0, 'original data')
x_filter, y_filter = field_log.get_values()
fig.plot(x_filter, y_filter, 'filtered data')
fig.show()


We can also check which data has been removed from the other sample log (Temp), but this time we will use the `plot_sample_log` method from the `Figure` object to make the code a bit cleaner

In [ ]:
fig = Figure(y_label='Temperature (Kelvin)', x_label='Time (seconds)')
fig.plot_from_sample_log(data.get_sample_log('Temp'))
fig.show()

If we add the wrong filter (e.g. typo), we can remove it with the command

In [ ]:
data.delete_sample_log_filter('field')

where the argument is the name of the filter to be deleted. To check that it worked, lets plot the histogram (generating/saving a histogram will update all of the data). 

In [ ]:
hist_check, bins = data.histogram()
fig = Figure(y_label='Counts')
fig.plot_from_histogram(bins, no_filter_hist, [0], 'unfiltered, ')
fig.plot_from_histogram(bins, hist_check, [0], 'Check filter removed, ')
fig.show()

## Sample log filters - Keeping data below a threshold value

The second type of filter we will look at removes all of the data above a sepcific value. For this example lets remove all of the data with a field value of greater than 10.

In [ ]:
data.keep_data_sample_log_below('field', 10.)

The `keep_data_sample_log_below` command is used to add a filter, the first argument is the sample log name we want to apply the filter to and the second argument is the maximum value we want to keep. Lets compare the resultant histogram with the unfiltered data. 


In [ ]:
hist_below_10, bins = data.histogram()
fig = Figure(y_label='Counts')
fig.plot_from_histogram(bins, no_filter_hist, [0], 'unfiltered, ')
fig.plot_from_histogram(bins, hist_above_6, [0], 'field <= 10, ')
fig.show()

Next lets plot the filtered sample log data;

In [ ]:
fig = Figure(y_label='Field (MHz)', x_label='Time (seconds)')
fig.plot_from_sample_log(data.get_sample_log('field'))
fig.show()


In [ ]:
fig = Figure(y_label='Temperature (Kelvin)', x_label='Time (seconds)')
fig.plot_from_sample_log(data.get_sample_log('Temp'))
fig.show()

Lets remove the filter,

In [ ]:
data.delete_sample_log_filter('field')

## Sample log filters - Keeping data within a range of values

The last type of sample log filter we will look at keeps data between two values. At present it is only possible to add one of these band filters per sample log. Lets start by adding the filter to the `Temp` sample log

In [ ]:
data.keep_data_sample_log_between('Temp', 2, 4.5)

The `keep_data_sample_log_between` command is used to add a filter that keeps the data between the two values. The first argument is the sample log name we want to apply the filter to, the second argument is the minimum value we want to keep and the final argument is the maximum value we want to keep. Lets compare the resultant histogram with the unfiltered data. 


In [ ]:
hist_band, bins = data.histogram()
fig = Figure(y_label='Counts')
fig.plot_from_histogram(bins, no_filter_hist, [0], 'unfiltered, ')
fig.plot_from_histogram(bins, hist_band, [0], 'Temp band, ')
fig.show()

Next lets plot the filtered sample log data;

In [ ]:
fig = Figure(y_label='Temperature (Kelvin)', x_label='Time (seconds)')
fig.plot_from_sample_log(data.get_sample_log('Temp'))
fig.show()


In [ ]:
fig = Figure(y_label='Field (MHz)', x_label='Time (seconds)')
fig.plot_from_sample_log(data.get_sample_log('field'))
fig.show()

Lets remove the filter,

In [ ]:
data.delete_sample_log_filter('Temp')